In [1]:
import cv2
import numpy as np
import os

def check_battle(results):
    global battle_start, start_time
    classes = {}
    for r in results:
        for c in r.boxes.cls:
            name = names[int(c)]
            if name in classes:
                classes[name] += 1
            else:
                classes[name] = 1
    if classes.get('Beyblade', 0) >= 2:
        start_time = cap.get(cv2.CAP_PROP_POS_MSEC)
        battle_start = 1
        print('Battle dimulai', start_time)

def create_region(frame, save=False, filename="region_1", folder="region"):  
    if type(frame) == str:
        frame = cv2.imread(frame)

    # Create a copy of the frame to draw on
    img = frame.copy()
    region = []

    # Define the callback function for mouse events
    def click_event(event, x, y, flags, params):
        nonlocal img, region
        if event == cv2.EVENT_LBUTTONDOWN:
            # Draw a circle on the image
            cv2.circle(img, (x, y), 5, (255, 255, 0) , -1)
            region.append(x)
            region.append(y)
            cv2.imshow("draw region", img)

    # Create a window to display the image
    cv2.namedWindow("draw region")

    # Set the mouse callback function for the window
    cv2.setMouseCallback("draw region", click_event, {"img": img})

    # Display the image and wait for a key press
    cv2.putText(  # Display instructions
        img,
        "Click to define region. Press 'f' to finish",
        (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255, 255, 255),
        2,
    )
    cv2.imshow("draw region", img)
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord("f"):
            if len(region) == 8:
                cv2.destroyAllWindows()
                return region
            else:
                print('Jumlah titik Koordinat yang harus anda masukkan adalah 4')
                region = []
                cv2.destroyAllWindows()
                cv2.namedWindow("draw region")
                img = frame.copy()
                cv2.setMouseCallback("draw region", click_event, {"img": img})

                cv2.putText(  # Display instructions
                    img,
                    "Click to define region. Press 'f' to finish and 'n' to next region",
                    (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 255, 255),
                    2,
                )

def draw_region(frame, region):
    vertices = np.array([[region[i], region[i+1]] for i in range(0, len(region), 2)])
    cv2.polylines(frame, [vertices], isClosed=True, color=(0, 255, 0), thickness=2)

def is_out(result, region):
    vertices = np.array([[region[i], region[i+1]] for i in range(0, len(region), 2)])
    bound_box = 0
    out_status = {}
    for r in results:
        cls = r.boxes.cls
        bound_box = r.boxes.xyxy
        for c in cls:
            name = names[int(c)]
            if name == 'Beyblade':
                for index, box in enumerate(bound_box):
                    x1, y1, x2, y2 = box.tolist()
                    xc = (x1 + x2) / 2
                    yc = (y1 + y2) / 2
                    entry = cv2.pointPolygonTest(vertices, (xc, yc), measureDist=False)
                    out_status[(xc,yc)] = {
                            'status' : 'menang',
                            'x1': x1,
                            'y1': y1,
                            'x2': x2,
                            'y2': y2,
                        }
                    if (entry == -1):
                        out_status[(xc,yc)]['status'] = 'kalah'

    return out_status

def check_spin_status(results, min_movement=1):
    spin_status = {}
    for r in results:
        for c in r.boxes.cls:
            name = names[int(c)]
            if name == 'Beyblade':
                box = r.boxes.xyxy
                for b in box:
                    x1, y1, x2, y2 = b.tolist()
                    xc = round((x1 + x2) / 2)  # Round to the nearest integer
                    yc = round((y1 + y2) / 2)  # Round to the nearest integer
                    spin_status[(xc, yc)] = {
                        'x1': x1,
                        'y1': y1,
                        'x2': x2,
                        'y2': y2, 
                        'status' : 'menang'
                    }

    # Calculate Optical Flow for spinning status
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if prev_frame_gray is not None:
        flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude = np.sqrt(flow[..., 0]**2 + flow[..., 1]**2)

        for pos, beyblade_info in spin_status.items():
            # Calculate movement around the Beyblade position
            x_min = max(0, int(pos[0]) - 5)
            x_max = min(magnitude.shape[1], int(pos[0]) + 5)
            y_min = max(0, int(pos[1]) - 5)
            y_max = min(magnitude.shape[0], int(pos[1]) + 5)
            movement = np.mean(magnitude[y_min:y_max, x_min:x_max])
            if movement < min_movement:
                spin_status[pos]['status'] = 'kalah'

    return spin_status

def is_taken(results):
    hand_coords = []
    beyblade_coords = []
    hand_detected = False

    # Identify the class index of Beyblade
    for r in results:
        for c in r.boxes.cls:
            name = names[int(c)]
            if name == 'Beyblade':
                beyblade_box = r.boxes.xyxy.tolist()
                for b in beyblade_box:
                    x1, y1, x2, y2 = b
                    beyblade_coords.append(((x1, y1), (x2, y2)))
            else:
                hand_box = r.boxes.xyxy.tolist()
                for b in hand_box:
                    x1, y1, x2, y2 = b
                    hand_coords.append(((x1, y1), (x2, y2)))
                hand_detected = True

    if not hand_detected:
        return False  # No hand detected, skip the Beyblade check

    # Check if any point in the hand box is inside the Beyblade box
    for hand_box in hand_coords:
        for beyblade_box in beyblade_coords:
            x1_hand, y1_hand = hand_box[0]
            x2_hand, y2_hand = hand_box[1]
            x1_bey, y1_bey = beyblade_box[0]
            x2_bey, y2_bey = beyblade_box[1]
            
            x_center = (x1_bey + x2_bey) / 2
            y_center = (y1_bey + y2_bey) / 2
            
            if x_center >= x1_hand and x_center <= x2_hand and y_center >= y1_hand and y_center <= y2_hand:
                return True

In [2]:
import cv2
from ultralytics import YOLO
import pandas as pd

# Load the YOLOv8 model
model = YOLO('best.pt')
names = model.names

# Open the video file
video_path = "beyblade.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties (if you dont want to save video, delete 6 code line below)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
output_path = "output_video.avi"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

battle_start = 0
start_time = 0
arena = None
reason = None
prev_frame_gray = None

# Loop through the video frames
while cap.isOpened():

    kondisi = {'kalah' : 0, 'menang' : 0}

    # Read a frame from the video
    success, frame = cap.read()
    
    if arena != None:
        draw_region(frame, arena)

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, verbose= False, tracker='botsort.yaml')
        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        if battle_start == 0:
            check_battle(results)
        else:
            time = cap.get(cv2.CAP_PROP_POS_MSEC)
            battle_time = (time - start_time) / 1000

            # check if the beyblade out from arena
            if arena != None:
                out_check = is_out(results, arena)
                for x, y in out_check.items():
                    status = y['status']
                    if status == 'kalah':
                        kondisi['kalah'] += 1
                        x1, y1, x2, y2 = y['x1'], y['y1'], y['x2'], y['y2']
                    else:
                        kondisi['menang'] += 1
                        x3, y3, x4, y4 = y['x1'], y['y1'], y['x2'], y['y2']
                    if kondisi['menang'] == 1 and kondisi['kalah'] == 1:
                        reason = 'Musuh keluar arena'
                        loser_out_path = 'loser.jpg'
                        winner_out_path = 'winner.jpg'
                        frame_lose = frame[int(y1):int(y2), int(x1):int(x2)]
                        frame_win = frame[int(y3):int(y4), int(x3):int(x4)]
                        cv2.imwrite(winner_out_path, frame_win)
                        cv2.imwrite(loser_out_path, frame_lose)
                        cv2.imwrite('kondisi_terakhir.jpg', frame)
                        cap.release()
                        break

            # check if the beyblade stop spinning
            kondisi = {'kalah' : 0, 'menang' : 0}
            spin_status = check_spin_status(results)
            for x, y in spin_status.items():
                status = y['status']
                if status == 'kalah':
                    kondisi['kalah'] += 1
                    x1, y1, x2, y2 = y['x1'], y['y1'], y['x2'], y['y2']
                else:
                    kondisi['menang'] += 1
                    x3, y3, x4, y4 = y['x1'], y['y1'], y['x2'], y['y2']
                if kondisi['menang'] == 1 and kondisi['kalah'] == 1:
                    reason = 'Musuh Berhenti berputar'
                    loser_out_path = 'loser.jpg'
                    winner_out_path = 'winner.jpg'
                    frame_lose = frame[int(y1):int(y2), int(x1):int(x2)]
                    frame_win = frame[int(y3):int(y4), int(x3):int(x4)]
                    cv2.imwrite(winner_out_path, frame_win)
                    cv2.imwrite(loser_out_path, frame_lose)
                    cv2.imwrite('kondisi_terakhir.jpg', frame)
                    cap.release()
                    break

            # check if the beyblade taken by hand
            if battle_time > 3 :
                hand_taken = is_taken(results)
                if hand_taken == True:
                    reason = 'Musuh diambil'
                    cv2.imwrite('kondisi_terakhir.jpg', frame)

        prev_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # save the video (delete 1 line code below if you dont want save the video).
        out.write(annotated_frame)

        # Break the loop if 'q' is pressed
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        elif key == ord("d"):
            arena = create_region(frame)
    else:
        break

data = {'battle_time' : battle_time,
        'winner' : winner_out_path,
        'loser' : loser_out_path,
        'reason' : reason,
       }

index = ['Battle Data']

df = pd.DataFrame(data, index=index)

cap.release()
cv2.destroyAllWindows()

df.to_csv('battle_result.csv', index=False)

Battle dimulai 2966.666666666667
